In [2]:
import argparse
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import tensorflow.lite as tflite
from tensorflow import keras
import zlib
from platform import python_version
import tensorflow_model_optimization as tfmot   
import tempfile
print(f"Python version used to excute the code is {python_version()}")



Python version used to excute the code is 3.7.5


In [3]:
from classes import read_audios
from classes import SignalGenerator
from classes import make_models
from classes import model_analysis
from classes import latency

In [4]:
version = "a"
m = "cnn"   # model name [ mlp , cnn , ds_cnn  ]
mfcc = True    # True --> excute mfcc , False --> excute STFT
alpha = 0.3    # The width multiplier used to apply the structured Pruning 
epochs = 20

model_version = f"_V_{version}_alpha={alpha}"
mymodel = m + model_version
TFLITE =  f'{mymodel}.tflite'     # path for saving the best model after converted to TF.lite model 
units = 8                         # The number of output class [8:without silence , 9 : with silence]
################## Fix the Random seed to reproduce the same results 
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [5]:
# zip_path = tf.keras.utils.get_file(
#     origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
#     fname='mini_speech_commands.zip',
#     extract=True,
#     cache_dir='.', cache_subdir='data')

# data_dir = os.path.join('.', 'data', 'mini_speech_commands')
# data_dir = 'E:\Github\Machine-learning-for-IOT\Lab3\ex2\data\mini_speech_commands'
# reading_class = read_audios(data_dir)
reading_class = read_audios()
train_files, val_files, test_files = reading_class.read()

In [6]:
alpha = [0.15, 0.3, 0.6, 1]
mfcc = [True]
m = ['cnn', 'ds_cnn']
number_of_bins = [16,40]
lower_freq = [20]
upper_freq = [4000]
sample_rate = [16000]
frame_length = [320, 640, 840, 1020]
frame_step = [128, 320, 640, 840]

# alpha = [0.3]
# mfcc = [True]
# m = ['cnn']
# number_of_bins = [16]
# lower_freq = [20]
# upper_freq = [4000]
# sample_rate = [16000]
# frame_length = [1000]
# frame_step = [100]
df = pd.DataFrame(columns=['alpha', 'mfcc', 'model', 'number_of_bins', 'lower_freq', 'upper_freq','sample_rate','frame_step', 'frame_length','acuracy','size','latency'])
for p_alpha in range(len(alpha)):
    for p_mfcc in range(len(mfcc)):
        for p_m in range(len(m)):
            for p_bins in range(len(number_of_bins)):
                for p_lower_f in range(len(lower_freq)):
                    for p_upper_f in range(len(upper_freq)):
                        for p_rate in range(len(sample_rate)):
                            for p_f_step in range(len(frame_step)):
                                for p_f_length in range(len(frame_length)):
                                    if((lower_freq[p_lower_f] < upper_freq[p_upper_f]) and (upper_freq[p_lower_f] <= sample_rate[p_upper_f]/2) and (frame_step[p_f_step] < frame_length[p_f_length])):
                                        data = [[alpha[p_alpha],mfcc[p_mfcc],m[p_m],number_of_bins[p_bins],lower_freq[p_lower_f],upper_freq[p_upper_f],sample_rate[p_rate],frame_step[p_f_step],frame_length[p_f_length],0,0,0]]
                                        d = pd.DataFrame(data, columns=['alpha', 'mfcc', 'model', 'number_of_bins', 'lower_freq', 'upper_freq','sample_rate','frame_step', 'frame_length','acuracy','size','latency'])
                                        df = df.append(d, ignore_index = True)

In [ ]:
LABELS = np.array(['stop', 'up', 'yes', 'right', 'left', 'no',  'down', 'go'] , dtype = str) 

for i in range(df.shape[0]):
    print("configuration  " + str(i) + " from  " + str(df.shape[0])+ "!!!!!!!!!!!!")
    model_version = f"_V_{version}_alpha={df.loc[i]['alpha']}"
    mymodel = df.loc[i]['model'] + model_version
    TFLITE =  f'{mymodel}.tflite' 

    STFT_OPTIONS = {'frame_length': df['frame_length'][i], 'frame_step': df['frame_step'][i], 'mfcc': False}
    MFCC_OPTIONS = { 'sampling_rate': df['sample_rate'][i],'frame_length': df['frame_length'][i], 'frame_step': df['frame_step'][i], 'mfcc': True,
            'lower_frequency': df['lower_freq'][i], 'upper_frequency': df['upper_freq'][i], 'num_mel_bins': df['number_of_bins'][i],
            'num_coefficients': 10}
    if df.loc[i]['mfcc'] is True:
        options = MFCC_OPTIONS
        strides = [2, 1]
    else:
        options = STFT_OPTIONS
        strides = [2, 2]

    generator = SignalGenerator(LABELS, **options)
    train_ds = generator.make_dataset(train_files, True)
    val_ds = generator.make_dataset(val_files, False)
    test_ds = generator.make_dataset(test_files, False)

    model_maker = make_models()
    ############ Applying Structured-Based Pruning
    model, model_checkpoint_callback, checkpoint_filepath = model_maker.models(df['alpha'][i], strides, units, model_version, df['mfcc'][i], mymodel,False,train_ds)

    ############ Applying Magnitude-Based Pruning
    #model, model_checkpoint_callback, checkpoint_filepath = model_maker.models(1, strides, units, model_version, mfcc, mymodel)
    history = model.fit(train_ds, epochs=epochs,   validation_data=val_ds,callbacks=[model_checkpoint_callback ], verbose=0)
    model_maker.plot_loss(history, mymodel)

    analysis = model_analysis(test_ds, checkpoint_filepath, train_ds)
    Compressed , tflite_model_dir = analysis.S_pruning_Model_evaluate_and_compress_to_TFlite( tflite_model_dir = TFLITE)

    acc, size = analysis.load_and_evaluation(tflite_model_dir, Compressed)

    laten = latency()
    inf, tot = laten.calculate(model = tflite_model_dir, mfcc = df['mfcc'][i] ,rate = df['sample_rate'][i], length = df['frame_length'][i], stride = df['frame_step'][i],lower_frequency = df['lower_freq'][i], upper_frequency = df['upper_freq'][i], num_mel_bins = df['number_of_bins'][i])
    print("acc =   " + str(acc))
    print("size =   " + str(size))
    print("latency =   " + str(tot))
    df['acuracy'][i] = acc
    df['size'][i] = size
    df['latency'][i] = tot

In [50]:
grid = pd.read_csv('grid_search_result.csv').drop(columns = ['Unnamed: 0'])

In [56]:
#grid.sort_values(by = ['acuracy'], ascending = False).head(20)
grid.loc[grid['size'] <= 500].loc[grid['acuracy'] >= 92].sort_values(by = ['acuracy'], ascending = False).reset_index().drop(['index'], axis=1)

,alpha,mfcc,model,number_of_bins,lower_freq,upper_freq,sample_rate,frame_step,frame_length,acuracy,size,latency
0,0.6,True,cnn,40,20,4000,16000,320,1020,94.000,393.991,43.244722
1,0.6,True,cnn,40,20,4000,16000,320,640,93.875,393.998,44.951603
2,0.6,True,ds_cnn,16,20,4000,16000,320,1020,93.875,197.615,40.674331
3,0.6,True,ds_cnn,16,20,4000,16000,320,640,93.875,197.698,42.707617
4,0.6,True,ds_cnn,16,20,4000,16000,320,840,93.500,197.682,42.175066
5,0.6,True,ds_cnn,16,20,4000,16000,128,1020,93.250,197.787,86.764703
6,0.3,True,cnn,16,20,4000,16000,320,1020,93.250,101.262,41.077759
7,0.3,True,ds_cnn,16,20,4000,16000,320,840,93.125,55.664,41.270542
8,0.6,True,cnn,40,20,4000,16000,128,840,93.125,394.143,92.651932
9,0.6,True,ds_cnn,40,20,4000,16000,320,840,93.000,197.595,43.749540


None


In [26]:
%run kws_latency.py --model ./models/cnn_V_a_alpha=0.3.tflite --mfcc --rate 16000 --lower-frequency 20 --upper-frequency 4000 --bins 40

Inference Latency 2.67ms
Total Latency 72.43ms


In [25]:
laten = latency()
inf, tot = laten.calculate(model = './models/cnn_V_a_alpha=0.3.tflite', mfcc = True ,rate = 16000, lower_frequency = 20, upper_frequency = 4000, num_mel_bins = 40)

Inference Latency 2.09ms
Total Latency 55.03ms


In [ ]:
Compressed , Quantized   = analysis.apply_Quantization(TFLITE, PQT=True , WAPQT = False)

the model is saved successfuly to ./models/PQT_cnn_V_a_alpha=0.3.tflite


In [ ]:
analysis.load_and_evaluation(Quantized , Compressed)

************************************************** 
 The Size of TF lite model  Before compression is = 32.992 kb
************************************************** 
 The Size of TF lite model  After compression is = 28.237 kb
************************************************** 
 The accuracy of TF lite model is = 69.62 


In [ ]:
WA_Compressed , WA_Quantized   = analysis.apply_Quantization(TFLITE, PQT=False ,WAPQT=True)

TypeError: 'generator' object is not callable

In [ ]:
analysis.load_and_evaluation(WA_Quantized , WA_Compressed)